In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings('ignore')

In [54]:
df = pd.read_csv('Data/Preprocessed.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.813399,148.0,72.0,0.696814,0.183531,0.260117,0.821764,1.364180,1
1,-0.833906,85.0,66.0,0.036596,0.183531,-0.843284,-0.168409,0.126452,0
2,1.188996,183.0,64.0,0.054048,0.183531,-1.457745,0.935284,0.230161,1
3,-0.833906,89.0,66.0,-0.674132,-0.791683,-0.586404,-1.298725,-1.480075,0
4,-1.603317,137.0,40.0,0.696814,0.343910,1.462682,2.336680,0.327328,1


In [55]:
features = df.drop('Outcome', axis=1)
label = df['Outcome']
print(features.shape, label.shape)

(768, 8) (768,)


In [123]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=63)

In [161]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=2, p=3, weights='distance')

accuracy = []
print('[INFO] KNN Model Performance...\n')
i = 0
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    knn_model.fit(x_train_fold, y_train_fold)
    accuracy.append(knn_model.score(x_test_fold, y_test_fold))
    
    print('[INFO] Performance on', i, 'fold')
    # model performance report
    y_pred = knn_model.predict(x_test_fold)
    print(knn_model.score(x_test_fold, y_test_fold))
    print(confusion_matrix(y_test_fold, y_pred))
    print(classification_report(y_test_fold, y_pred))
    print('--'*30, '\n')
    i += 1
    
print('All accuracy', accuracy)
print('Max:', max(accuracy))
print('Average', np.mean(accuracy))

[INFO] KNN Model Performance...

[INFO] Performance on 0 fold
0.6688311688311688
[[77 23]
 [28 26]]
              precision    recall  f1-score   support

           0       0.73      0.77      0.75       100
           1       0.53      0.48      0.50        54

    accuracy                           0.67       154
   macro avg       0.63      0.63      0.63       154
weighted avg       0.66      0.67      0.66       154

------------------------------------------------------------ 

[INFO] Performance on 1 fold
0.6493506493506493
[[70 30]
 [24 30]]
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       100
           1       0.50      0.56      0.53        54

    accuracy                           0.65       154
   macro avg       0.62      0.63      0.62       154
weighted avg       0.66      0.65      0.65       154

------------------------------------------------------------ 

[INFO] Performance on 2 fold
0.6168831168831169
[[70 

In [167]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()

accuracy = []
print('[INFO] Logistic Regression Model Performance...\n')
i = 0
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    lr_model.fit(x_train_fold, y_train_fold)
    accuracy.append(lr_model.score(x_test_fold, y_test_fold))
    
    print('[INFO] Performance on', i, 'fold')
    # model performance report
    y_pred = lr_model.predict(x_test_fold)
    print(lr_model.score(x_test_fold, y_test_fold))
    print(confusion_matrix(y_test_fold, y_pred))
    print(classification_report(y_test_fold, y_pred))
    print('--'*30, '\n')
    i += 1
    
print('All accuracy', accuracy)
print('Max:', max(accuracy))
print('Average', np.mean(accuracy))

[INFO] Logistic Regression Model Performance...

[INFO] Performance on 0 fold
0.7337662337662337
[[84 16]
 [25 29]]
              precision    recall  f1-score   support

           0       0.77      0.84      0.80       100
           1       0.64      0.54      0.59        54

    accuracy                           0.73       154
   macro avg       0.71      0.69      0.69       154
weighted avg       0.73      0.73      0.73       154

------------------------------------------------------------ 

[INFO] Performance on 1 fold
0.7727272727272727
[[87 13]
 [22 32]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       100
           1       0.71      0.59      0.65        54

    accuracy                           0.77       154
   macro avg       0.75      0.73      0.74       154
weighted avg       0.77      0.77      0.77       154

------------------------------------------------------------ 

[INFO] Performance on 2 fold
0.714285

In [166]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, n_jobs=-1, random_state=21)

accuracy = []
print('[INFO] Random Forest Model Performance...\n')
i = 0
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    rf_model.fit(x_train_fold, y_train_fold)
    accuracy.append(rf_model.score(x_test_fold, y_test_fold))
    
    print('[INFO] Performance on', i, 'fold')
    # model performance report
    y_pred = rf_model.predict(x_test_fold)
    print(rf_model.score(x_test_fold, y_test_fold))
    print(confusion_matrix(y_test_fold, y_pred))
    print(classification_report(y_test_fold, y_pred))
    print('--'*30, '\n')
    i += 1
    
print('All accuracy', accuracy)
print('Max:', max(accuracy))
print('Average', np.mean(accuracy))

[INFO] Random Forest Model Performance...

[INFO] Performance on 0 fold
0.7077922077922078
[[84 16]
 [29 25]]
              precision    recall  f1-score   support

           0       0.74      0.84      0.79       100
           1       0.61      0.46      0.53        54

    accuracy                           0.71       154
   macro avg       0.68      0.65      0.66       154
weighted avg       0.70      0.71      0.70       154

------------------------------------------------------------ 

[INFO] Performance on 1 fold
0.7857142857142857
[[83 17]
 [16 38]]
              precision    recall  f1-score   support

           0       0.84      0.83      0.83       100
           1       0.69      0.70      0.70        54

    accuracy                           0.79       154
   macro avg       0.76      0.77      0.77       154
weighted avg       0.79      0.79      0.79       154

------------------------------------------------------------ 

[INFO] Performance on 2 fold
0.733766233766

In [164]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', degree=20, random_state=0)

accuracy = []
print('[INFO] SVM Model Performance...\n')
i = 0
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    svm_model.fit(x_train_fold, y_train_fold)
    accuracy.append(svm_model.score(x_test_fold, y_test_fold))
    
    print('[INFO] Performance on', i, 'fold')
    # model performance report
    y_pred = svm_model.predict(x_test_fold)
    print(svm_model.score(x_test_fold, y_test_fold))
    print(confusion_matrix(y_test_fold, y_pred))
    print(classification_report(y_test_fold, y_pred))
    print('--'*30, '\n')
    i += 1
    
print('All accuracy', accuracy)
print('Max:', max(accuracy))
print('Average', np.mean(accuracy))

[INFO] SVM Model Performance...

[INFO] Performance on 0 fold
0.7142857142857143
[[82 18]
 [26 28]]
              precision    recall  f1-score   support

           0       0.76      0.82      0.79       100
           1       0.61      0.52      0.56        54

    accuracy                           0.71       154
   macro avg       0.68      0.67      0.67       154
weighted avg       0.71      0.71      0.71       154

------------------------------------------------------------ 

[INFO] Performance on 1 fold
0.7792207792207793
[[87 13]
 [21 33]]
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       100
           1       0.72      0.61      0.66        54

    accuracy                           0.78       154
   macro avg       0.76      0.74      0.75       154
weighted avg       0.77      0.78      0.77       154

------------------------------------------------------------ 

[INFO] Performance on 2 fold
0.7337662337662337
[[85 

In [165]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=200, random_state=0, eval_metric='mlogloss')

accuracy = []
print('[INFO] XGBoost Model Performance...\n')
i = 0
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    xgb_model.fit(x_train_fold, y_train_fold)
    accuracy.append(xgb_model.score(x_test_fold, y_test_fold))
    
    print('[INFO] Performance on', i, 'fold')
    # model performance report
    y_pred = xgb_model.predict(x_test_fold)
    print(xgb_model.score(x_test_fold, y_test_fold))
    print(confusion_matrix(y_test_fold, y_pred))
    print(classification_report(y_test_fold, y_pred))
    print('--'*30, '\n')
    i += 1
    
print('All accuracy', accuracy)
print('Max:', max(accuracy))
print('Average', np.mean(accuracy))

[INFO] XGBoost Model Performance...

[INFO] Performance on 0 fold
0.7207792207792207
[[81 19]
 [24 30]]
              precision    recall  f1-score   support

           0       0.77      0.81      0.79       100
           1       0.61      0.56      0.58        54

    accuracy                           0.72       154
   macro avg       0.69      0.68      0.69       154
weighted avg       0.72      0.72      0.72       154

------------------------------------------------------------ 

[INFO] Performance on 1 fold
0.8116883116883117
[[82 18]
 [11 43]]
              precision    recall  f1-score   support

           0       0.88      0.82      0.85       100
           1       0.70      0.80      0.75        54

    accuracy                           0.81       154
   macro avg       0.79      0.81      0.80       154
weighted avg       0.82      0.81      0.81       154

------------------------------------------------------------ 

[INFO] Performance on 2 fold
0.7142857142857143
[

<span style="color: yellow;">__*Insights*__</span>
* `SVM` is the best performer out of all with highest accuracy of 84%
* Out of all models we tried, only `Logistic Regression`, `Random Forest`, `SVM` and `XGBoost` are giving accuracy above 80%

In [143]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score